In [1]:
from bs4 import BeautifulSoup
import numpy as np
from nltk.tokenize import TweetTokenizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
f = open('paraphrases.xml', 'r')
content = f.read()

In [3]:
soup = BeautifulSoup(content)
tknzr = TweetTokenizer()

In [4]:
paraphrases = soup.findAll("paraphrase")
print(paraphrases[0])

<paraphrase>
<value name="id">1</value>
<value name="id_1">201</value>
<value name="id_2">8159</value>
<value name="text_1">Полицейским разрешат стрелять на поражение по гражданам с травматикой.</value>
<value name="text_2">Полиции могут разрешить стрелять по хулиганам с травматикой.</value>
<value name="jaccard">0.65</value>
<value name="class">0</value>
</paraphrase>


In [5]:
russian_baselines = {}
english_baselines = {}

with open('attract/baselines/en-ru.txt', 'r') as f:
    strings = f.readlines()
    for s in strings:
        parts = s.split()
        if parts[0].startswith("en_"):
            english_baselines[parts[0][3:]] = np.array([float(part) for part in parts[1:]], dtype=float)
        elif parts[0].startswith("ru_"):
            russian_baselines[parts[0][3:]] = np.array([float(part) for part in parts[1:]], dtype=float)

In [6]:
print(len(russian_baselines.keys()))

252905


In [36]:
print(len(english_baselines.keys()))

76843


In [9]:
list(russian_baselines.keys())[:1000]

['ординарный',
 'измолот',
 'хошэн',
 'лысак',
 'пипенция',
 'закоптелый',
 'какач',
 'ярвет',
 'полуразрушенный',
 'смать',
 'затеваться',
 'шаповал',
 'мэнсона',
 'супер',
 'эшшольция',
 'рабиология',
 'медиобазальный',
 'фонологически',
 'неказисто',
 'перстнещитовидный',
 'депрессивность',
 'ошеломление',
 'каверза',
 'раскупоривать',
 'арсеникум',
 'булавина',
 'перехватиться',
 'живопырка',
 'перетяткович',
 'семитизм',
 'гуделка',
 'мартыновский',
 'геоэкологический',
 'кустарничок',
 'суперструна',
 'кутырев',
 'саликов',
 'парфенков',
 'полнорационный',
 'сюзанн',
 'экстраполировать',
 'нераспорядительность',
 'проческа',
 'манталинь',
 'чухинский',
 'форвардер',
 'ретортный',
 'круговерть',
 'подрожать',
 'пфуй',
 'фиксатор',
 'затаиваться',
 'архипастырский',
 'колыхнуть',
 'металлоорганический',
 'вккэ',
 'дерма',
 'промсити',
 'террасер',
 'раздирать',
 'смокинг',
 'своровать',
 'четырехтонный',
 'оительст',
 'корячка',
 'киан',
 'возомнить',
 'татаринова',
 'анскулинг',
 

In [35]:
russian_baselines['пиздец']

array([-1.12300e-03, -3.29820e-02, -5.54700e-03,  2.59900e-03,
       -2.81690e-02, -6.52950e-02,  4.64900e-03, -7.01370e-02,
       -2.22360e-02, -1.42007e-01,  1.70236e-01, -5.63800e-03,
        2.59600e-03, -7.20880e-02,  3.68860e-02, -8.63460e-02,
        6.74580e-02, -8.67680e-02, -1.48800e-03,  1.78430e-02,
       -3.31630e-02, -5.78400e-03, -3.87500e-03,  7.66580e-02,
        9.15880e-02, -1.20301e-01, -6.06720e-02,  9.98900e-03,
       -1.68600e-02,  5.80610e-02,  1.02700e-01, -4.32750e-02,
       -3.45460e-02,  1.57800e-03, -7.75300e-03, -1.12314e-01,
        9.78830e-02,  1.23080e-02, -1.06100e-03,  7.31380e-02,
        3.12050e-02,  3.03640e-02, -2.39380e-02,  8.59150e-02,
        4.54100e-03, -5.10390e-02,  4.71980e-02, -5.94960e-02,
        2.56510e-02,  1.12280e-02,  9.34900e-03,  1.17410e-02,
        2.96580e-02,  6.66630e-02, -5.94060e-02,  2.68430e-02,
        3.97290e-02, -3.53710e-02, -1.15030e-02, -1.57440e-02,
       -6.48300e-02, -2.74900e-02, -9.21820e-02, -7.564

In [6]:
russian_tuned = {}
english_tuned = {}

with open('attract/results/final_vectors.txt', 'r') as f:
    strings = f.readlines()
    for s in strings:
        parts = s.split()
        if parts[0].startswith("en_"):
            english_tuned[parts[0][3:]] = np.array([float(part) for part in parts[1:]], dtype=float)
        elif parts[0].startswith("ru_"):
            russian_tuned[parts[0][3:]] = np.array([float(part) for part in parts[1:]], dtype=float)

In [28]:
treshold = 0.9

In [33]:
y_theory = []
y_pred_base = []
y_pred_tuned = []
num_sent = 0
for par in paraphrases:
    s1_words = tknzr.tokenize(par.findAll("value", {"name": "text_1"})[0].text.lower())
    s2_words = tknzr.tokenize(par.findAll("value", {"name": "text_2"})[0].text.lower())
    sim = int(par.findAll("value", {"name": "class"})[0].text)
    
    s1_emb_baseline = [0] * 300
    c1_baseline = 0
    s2_emb_baseline = [0] * 300
    c2_baseline = 0
    s1_emb_tuned = [0] * 300
    c1_tuned = 0
    s2_emb_tuned = [0] * 300
    c2_tuned = 0
    for w in s1_words:
        if w in russian_baselines.keys():
            s1_emb_baseline += russian_baselines[w]
            c1_baseline += 1
            s1_emb_tuned += russian_tuned[w]
            c1_tuned += 1
    for w in s2_words:
        if w in russian_baselines.keys():
            s2_emb_baseline += russian_baselines[w]
            c2_baseline += 1
            s2_emb_tuned += russian_tuned[w]
            c2_tuned += 1
            
    if c1_baseline != 0 and c1_tuned != 0 and c2_baseline != 0 and c2_tuned != 0:
        num_sent += 1
        s1_emb_baseline = np.array(s1_emb_baseline) / c1_baseline
        s1_emb_tuned = np.array(s1_emb_tuned) / c1_tuned
        s2_emb_baseline = np.array(s2_emb_baseline) / c2_baseline
        s2_emb_tuned = np.array(s2_emb_tuned) / c2_tuned
        cosine_base = cosine_similarity(np.array(s1_emb_baseline).reshape(1, -1), s2_emb_baseline.reshape(1, -1))
        cosine_tuned = cosine_similarity(s1_emb_tuned.reshape(1, -1), s2_emb_tuned.reshape(1, -1))
        if sim == 1:
            y_theory.append(sim)
        else:
            y_theory.append(0)
        if cosine_base < treshold:
            y_pred_base.append(0)
        else:
            y_pred_base.append(1)
        if cosine_tuned < treshold:
            y_pred_tuned.append(0)
        else:
            y_pred_tuned.append(1)

In [34]:
from sklearn.metrics import f1_score
print(f1_score(y_theory, y_pred_base))
print(f1_score(y_theory, y_pred_tuned))

0.37397099176793414
0.37465564738292007


In [21]:
num_success_baseline = 0
num_success_tuned = 0
num_sent = 0
for par in paraphrases:
    s1_words = tknzr.tokenize(par.findAll("value", {"name": "text_1"})[0].text.lower())
    s2_words = tknzr.tokenize(par.findAll("value", {"name": "text_2"})[0].text.lower())
    sim = int(par.findAll("value", {"name": "class"})[0].text)
    s1_emb_baseline = [0] * 300
    c1_baseline = 0
    s2_emb_baseline = [0] * 300
    c2_baseline = 0
    s1_emb_tuned = [0] * 300
    c1_tuned = 0
    s2_emb_tuned = [0] * 300
    c2_tuned = 0
    for w in s1_words:
        if w in russian_baselines.keys():
            s1_emb_baseline += russian_baselines[w]
            c1_baseline += 1
            s1_emb_tuned += russian_tuned[w]
            c1_tuned += 1
    for w in s2_words:
        if w in russian_baselines.keys():
            s2_emb_baseline += russian_baselines[w]
            c2_baseline += 1
            s2_emb_tuned += russian_tuned[w]
            c2_tuned += 1
            
    if c1_baseline != 0 and c1_tuned != 0 and c2_baseline != 0 and c2_tuned != 0:
        num_sent += 1
        s1_emb_baseline = np.array(s1_emb_baseline) / c1_baseline
        s1_emb_tuned = np.array(s1_emb_tuned) / c1_tuned
        s2_emb_baseline = np.array(s2_emb_baseline) / c2_baseline
        s2_emb_tuned = np.array(s2_emb_tuned) / c2_tuned

        cosine_base = cosine_similarity(np.array(s1_emb_baseline).reshape(1, -1), s2_emb_baseline.reshape(1, -1))
        if cosine_base > 0.05 and (sim == -1 or sim == 0):
            num_success_baseline += 1
        elif cosine_base <= 0.05 and sim == 1:
            num_success_baseline += 1
#         elif cosine_base < 0.3 and sim == 1:
#             num_success_baseline += 1
        cosine_tuned = cosine_similarity(s1_emb_tuned.reshape(1, -1), s2_emb_tuned.reshape(1, -1))
        if cosine_tuned > 0.05 and (sim == -1 or sim == 0):
            num_success_tuned += 1
        elif cosine_tuned <= 0.05 and sim == 1:
            num_success_tuned += 1
#         elif cosine_tuned < 0.3 and sim == -1:
#             num_success_tuned += 1    

    
            

In [22]:
print(num_success_baseline)
print(num_success_tuned)
print(num_sent)
print(len(paraphrases))

4016
4056
5912
7227


In [33]:
values = paraphrases[0].findAll("value", {"name": "text_1"})
print(values[0].text)

Цены на нефть восстанавливаются
